## Introduction

[CodeXGLUE](https://github.com/microsoft/CodeXGLUE) has recently been released by Microsoft Research Asia, Developer Division, and Bing. In their own words CodeXGLUE is 

> A benchmark dataset and open challenge for code intelligence. It includes a collection of code intelligence tasks and a platform for model evaluation and comparison. CodeXGLUE stands for General Language Understanding Evaluation benchmark for CODE. It includes 14 datasets for 10 diversified code intelligence tasks

You can think of it as something similar to [ImageNet](http://image-net.org/) for Computer Vision or [GLUE](https://gluebenchmark.com/) for NLP. 

This dataseet consists of several previously released one. Such as GitHub [CodeSearchNet](https://github.com/github/CodeSearchNet), [Py150](https://eth-sri.github.io/py150) and many more. CodeXGLUE also includes some baseline implementation and you can check them all out in their official Github repo. 

So far so good!

**BUT**

What if you want to ammase your own Data Set. Those datasets that we mentioned before were all created from Open Source repos, over the past few years. They are important and very valuable. But they do not, and will not, reflect your very own code-base and its very own, and specific twists. 

## Enter tree-hugger

We, at CodistAI, has recently open sourced our very own code processing library [tree-hugger](https://github.com/autosoft-dev/tree-hugger) (And we recently corssed 12K installations 🥁)

In this small tutorial we will show you how you can use tree-hugger to create your very own dataset from your very own code files which is very similar to the Open Source Data Set supplied by CodeXGLUE 

One reason for doing this can be to test the base line model in your own data and see how it performs (We are about to release `docly` a small command line tool which helps you to write function documentation for your Python code and we use the same parsing technique there as well 😀 )

### Let's install tree-hugger

In [1]:
!pip install -U tree-hugger PyYAML

     |████████████████████████████████| 276kB 4.0MB/s 
     |████████████████████████████████| 3.0MB 12.5MB/s 
     |████████████████████████████████| 112kB 37.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=210cd124adfa8c3f0b0cc9b68589f75baa32c717349725d3adae72d671fe116a
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for tree-sitter: filename=tree_sitter-0.2.0-cp36-cp36m-linux_x86_64.whl size=297395 sha256=5a642588b6c6967819047de6126a183ece612cea885b1436e6ac4a4d233a44cd
  Stored in directory: /root/.cache/pip/wheels/3b/a6/01/2290cc8441301a07e7717e9e03c6bbc0388f71c6bf1f7f37c1
Successfully built PyYAML tree-sitter
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### And use this command to build the necessary processing libary

In [2]:
!create_libs -c python

2020-10-09 13:28:58,710 INFO:Cloneing python repo from tree-sitter collections
2020-10-09 13:29:10,095 INFO:Creating the library my-languages.so at /content
2020-10-09 13:29:10,978 INFO:Finished creating library!


Now that we have all the necessary set-up done, let's download some files. For the ease of the tutorial we have created a small github example repo with a collection of files. Some of it is coming from Open Source repos and some we created as example files. 

Let's clone that

In [3]:
!git clone https://github.com/autosoft-dev/example-files.git

Cloning into 'example-files'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 2), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


We are going to declare a small function that will help us go over each files in a nested directory tree (like the one above we cloned) and get each file at a time. 

In [4]:
from pathlib import Path

def check_out_path(target_path: Path):
    """"
    This function recursively yields all contents of a pathlib.Path object
    """
    yield target_path
    for file in target_path.iterdir():
        if file.is_dir():
            yield from check_out_path(file)
        else:
            yield file.absolute()


def is_python_file(file_path: Path):
  """
  This little function will help us to filter the result and keep only the python files
  """
  return file_path.is_file() and file_path.suffix == ".py"

In [5]:
for file_path in check_out_path(Path("example-files")):
  if is_python_file(file_path):
    print(file_path)

/content/example-files/flask_files/cli.py
/content/example-files/inner_dir/_internal_utils.py
/content/example-files/api.py
/content/example-files/simple_funcs/simple_funcs.py


And now, we will define another small function, which, given a string which represents Python code will tokeize that

In [6]:
from tokenize import tokenize
from io import BytesIO


def tokenize_code_string(text):
    code_tokens = []
    for tok in tokenize(BytesIO(text.encode('utf-8')).readline):
        if tok.string.strip() != "" and tok.string.strip() != "utf-8":
            code_tokens.append(tok.string.strip().lower())
    return code_tokens

That is all for the pre-processing. Let's use tree-hugger's powerful API in conjunction with those functions to define a dataset from those custom files

In [7]:
# We first create our PythonParser object
from tree_hugger.core import PythonParser

In [8]:
pp = PythonParser(library_loc="/content/my-languages.so")

In [22]:
# We will now define a dict and populate it with the necessary data in a for loop

final_out_put_data = {}

for file_path in check_out_path(Path("example-files")):
  if is_python_file(file_path):
    final_out_put_data[file_path.stem] = None
    # we use one line, super convinient tree-hugger API call to get the needed data
    if pp.parse_file(str(file_path)):
      temp_cache = []
      # The following call returns a dict where each key is a name of a function
      # And each value is a tuple, (function_body, function_docstring)
      func_and_docstr = pp.get_all_function_bodies(strip_docstr=True)
      for func_name, (body, docstr) in func_and_docstr.items():
        code_tokens = tokenize_code_string(body)
        # Let's strip out all the internal comments
        final_code_tokens = [t for t in code_tokens if not t.startswith("#")]
        # Split the first line of docstring and remove all the tripple quotes and strip white spaces and make it lower
        docstr_tokens = docstr.split("\n")[0].strip().replace('"""', '').replace("'''", "").lower().split()
        temp_cache.append({"code": final_code_tokens, "docstr": docstr_tokens})
      # Let's add the result to the final output
      final_out_put_data[file_path.stem] = temp_cache

In [23]:
# And we are DONE!

final_out_put_data["api"][0]

{'code': ['def',
  'request',
  '(',
  'method',
  ',',
  'url',
  ')',
  ':',
  'with',
  'sessions',
  '.',
  'session',
  '(',
  ')',
  'as',
  'session',
  ':',
  'return',
  'session',
  '.',
  'request',
  '(',
  'method',
  '=',
  'method',
  ',',
  'url',
  '=',
  'url',
  ',',
  '**',
  'kwargs',
  ')'],
 'docstr': ['constructs',
  'and',
  'sends',
  'a',
  ':class:`request',
  '<request>`.']}

With this code, you can very easily create a dataset out of your own code files and then test the baseline models against it. 


That was easy!